In [1]:
"""
Visualization of single neuron dynamics for various models
"""
import numpy as np
import matplotlib.pyplot as plt
import neuron


In [16]:
Y0mpars={"P": 0.9, "gamma": 1e-1, "kappa": 2, "beta": 1e-2 }
#use completely random initial state
Y0params={"model" : "Yamada_0", "y0": np.random.random(2) , "dt": 1.e-6}

Y0Neuron=neuron.Neuron(Y0params)
Y0Neuron.fun(0, [0, 0])

NameError: name 'np' is not defined